# Magnetic Field, Thermal Field and Electrical Circuit Modeling of a Distribution Transformer

## Domenico Lahaye (TU Delft) and Jeroen Schuddebeurs (STEDIN Rotterdam)

## Questions
2. what are temperature ranges for different parts of the transformer in normal and non-normal operation point? 
3. ? 

## Section 1:/ Introduction 

Motivation - asset managnememt - problem sketch - approach - images - see text Max.   

## Section 2:/ 2D Magnetic Model to Estimate Coil and Core Losses  

Resonable estimates for the losses and temperature in various parts of the transformer would be good to have. 

### Literature Study as a Start on the Assignment 

### 2D Geometry Definition
We wish to obtain a parametric CAD model. Introduce depth perpendicular to modeling plane. Cfr. work of Gijs Lagerweij. 

### 2D Mesh Generation 
1. Structured mesh or boundary refinement to capture eddy current effects in the core;
2. Mesh the coil cross sections using rectangular elements and mesh remainder of geometry using triangular elements; 

### 2D Time-Harmonic Magnetic Model in Terms of the Magnetic Vector Potential Component A$_z$

1. take saturation curve into account; high frequency will result in small skin-depth with large values of magnetic flux inside a small layer; without saturation curve, normB and losses are likely to be overestimated;   
2. take electrical conductivity and magnetic permeability of the lamination into account; lower the values for ferromagnetic material by some factor; this factor is hard to estimate; what is known in literature? 
3. extend to second order elements; does type of element in mesh generation require adaption?   

### 2D Post-Processing for Losses in the Time-Harmonic Magnetic Model 
1. losses in the winding: [copper losses](https://en.wikipedia.org/wiki/Copper_loss); determined sperimentally using [short-circuit test](https://www.electricaleasy.com/2014/04/open-and-short-circuit-test-on-transformer.html); 
2. losses in the ferromagnetic core: [core loss](https://en.wikipedia.org/wiki/Magnetic_core#Core_loss): hysteresis losses and eddy-current losses; [open-circuit test](https://www.electricaleasy.com/2014/04/open-and-short-circuit-test-on-transformer.html); using Steinmetz formula; material dependent parameters might be difficult to recover; 
3. both copper losses and core losses expressed in W/m^3 such that value over entire volume is expressed in W;
4. run simulation of short-circuit and open-circuit test for a set of increasing frequencies, post-process for skin depth in the ferromagnetic core along a vertical line, post-process for relative importance of coil and core losses; 

## Section 3:/ 2D Thermal Model in Post-Processing of Magnetic Model 

### Literature Study as a Start on the Assignment 

### 2D Geometry Definition and Mesh Generation as Before  

1. document thermal conductivity, density and heat capacity of coil, core, insulating material and air domain;
2. compute thermal field; how to limit computational domain? What boundary conditions should be imposed at the boundaries;
3. how to take the presence of the insulating oil into account; break-down on oil due to overheating; 
4. how to take laminar/turbulent flow of oil due to buoyancy into account; using fluid dynamics?  
5. repeat for various values of the convective contribution; 

### Temperature Dependence of Magnetic Parameters and 2D Coupled Magnetic-Thermal Model
1. temperature dependence of [thermal conductivity](https://en.wikipedia.org/wiki/Electrical_resistivity_and_conductivity#Temperature_dependence);  
2. temperature dependence of [magnetic permeability](https://www.sentesoftware.co.uk/mint-project/uploads/850554056.pdf); 
3. gridap.jl supports modeling of coupled fields; how to take electrical circuit equations into account? 

### Electrical Circuit Model 
1. more complex representation of the driving electrical circuit; allow for single phase and three phase electrical circuit; 

### 2D Transient Model in Terms of the Magnetic Vector Potential Component A$_z$

1. extend from frequency domain to time-domain 

## Section 4:/ 2D and 3D Thermal Model to Estimate Temperature 

Estimate losses from literature or approximate models (loading guide). Compute temperature distribution with and without oil container. Measured temperature on lateral surface of the oil container is max 200 degrees Celcius. Given no information on largest temperature inside of domain and duration of this temperature. Properties of [transformer oil](https://en.wikipedia.org/wiki/Transformer_oil). 

## Section 5:/ 3D Geometry Definition and Mesh Generation 

1. geometry: core by extrusion in depth direction; (use external CAD tool); 
2. geometry: coils by extrusion in height direction; 
2. lamination of the ferromagnetic core; 
3. gridap.jl supports 3D edge elements for the magnetic field;

##  Section 6:/ 3D Thermal Model Assuming Given Core/Coil Loss Distribution 

Solve scalar field assuming loss computation. 

##  Section 7:/ 3D Magnetic Model

Solve vector potential. 

### 3D Nedelec Elements 

Example from @DaniStauber taken from [Gridap Gitter](https://gitter.im/Gridap-jl/community)


### To take into account 
1. sparse linear algebra (Krylov and algebraic multigrid) solvers for large scale (2D linear and non-linear) problems; AMG as preconditioner for CG for diffusion problems; preconditioned GMRES for all other problems;

## Section 8:/ Work in progress 
Work in progress on the 2D model by [Gijs Lagerweij]( https://github.com/gijswl/ee4375_fem_ta/blob/main/assignment2_fem2d/Distribution%20Transformer%20Stedin.ipynb). 

## References 
- Master thesis [Max van Dijk](https://repository.tudelft.nl/islandora/object/uuid%3A15b25b42-e04b-4ff2-a187-773bc170f061?collection=education)
- Comsol Multiphysics [Computing Losses in a Three Phase Transformer](https://www.comsol.com/blogs/computing-losses-in-a-three-phase-power-transformer).

In [ ]:
using Gridap
using GridapODEs
using GridapMakie, GLMakie
using LinearAlgebra
using FileIO
using GridapGmsh
using gmsh

model = GmshDiscreteModel("modelos/cubo_esfera_tet.msh")

Ω = Triangulation(model)
degree = 3
dΩ = Measure(Ω,degree)

B0=1.
A0_x(x)=-(B0/2.)*x[2]
A0_y(x)=(B0/2.)*x[1]

dirichlet_tags=["ext"]
dirichlet_values(x) = VectorValue(A0_x(x),A0_y(x),0.0)


order = 0
reffe = ReferenceFE(nedelec,Float64,order)
V = FESpace(model, reffe, conformity=:HCurl, dirichlet_tags=dirichlet_tags)

U= TrialFESpace(V,dirichlet_values)

f=VectorValue(1.,1.,1.)# #Source

a(u,v) =  ∫( (∇ × v)⋅(∇ × u))*dΩ
b(v) =  ∫(f ⋅v)dΩ 

op = AffineFEOperator(a,b,U,V)

ls = LUSolver()
solver = LinearFESolver(ls)

uh, = solve(solver,op)